# **1-1. 정신건강 분류 모델 학습**

---
> Ai Hub에서 제공하는 '웰니스 대화 스크립트 데이터셋'을 기반으로 내담자의 정신건강을 아래와 같은 3개의 단계로 구분하는 Multiclass Classification 모델을 학습합니다.



1.   **우울(Depression)**
2.   **초조(Anxiety)**
3.   **공통(위험군)(Both)**



> Self Attentive Sentence Embedding 기법을 응용하여 설문조사의 11가지 문항별로 화자에게 나타나는 정신증상을 분류합니다.

> 문항별로 (우울, 공통(위험군), 초조)의 확률이 Softmax에 의해 반환되며, 이를 플랫폼에 시각화합니다.
---

In [ ]:
# 드라이브 내 Custom Module 및 .ipynb 파일, 그리고 학습 데이터를 저장한 Directory를 입력하세요.

DIRECTORY = "AI경진대회" # 여기를 변경하세요.

## **(1) 라이브러리 준비**

In [ ]:
# Google Colab을 기반으로 학습을 진행할 경우, BERT 계열의 모델을 사용하기 위해 필요한 라이브러리를 설치합니다.

!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install torchmetrics

In [ ]:
import torch
import time
import os
import pickle
import random
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import gluonnlp as nlp
import matplotlib.pyplot as plt
from tqdm import tqdm
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertTokenizer, BertModel, AutoModel, AutoTokenizer
from kobert import get_pytorch_kobert_model

## **(2) Drive Mount 및 Custom Module 불러오기**

In [ ]:
# Drive Mount
from google.colab import drive
drive.mount('/content/gdrive')

# Directory 변경
path = "/content/gdrive/My Drive/" + DIRECTORY
os.chdir(path)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from dataset import WellSet
from model import AttLSTM
from train import train_model

## **(3) 데이터셋 불러오기**

In [ ]:
# 데이터셋 준비

with open('./DATA/1.정신건강/Wellness_Script.pkl', 'rb') as f:
    data = pickle.load(f)

data.head()

In [ ]:
# Train / Test 분리

index = list(data.index)
random.Random(1398).shuffle(index)
train_idx = index[:(round(len(index) * 0.8))]
test_idx = index[(round(len(index) * 0.8)):]

train = data.iloc[train_idx, :]
test = data.iloc[test_idx, :]

train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

## **(4) KcELECTRA, Tokenizer 불러오기**

In [ ]:
# KcELECTRA, Tokenizer

tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base-v2022")
kc_model = AutoModel.from_pretrained("beomi/KcELECTRA-base-v2022")

## **(5) DataLoader 준비**

In [ ]:
# Dataset

train_set = WellSet(tokenizer=tokenizer, text_column='Text', data=train, shuffle=False)
valid_set = WellSet(tokenizer=tokenizer, text_column='Text', data=test, shuffle=False)

In [ ]:
# DataLoader

BATCH_SIZE=128

train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

## **(6) 모델 생성**

In [ ]:
# Model (AttLSTM)

model = AttLSTM(num_layers=2,
                embedding_size=768,
                hidden_size=384,
                attention_dim=512,
                keys=30,
                fc_dim=512,
                batch_size=BATCH_SIZE)

## **(7) 모델 학습**

In [ ]:
# Train Model

model_aft, train_loss, valid_loss = train_model(train_dataloader=train_loader,
                                                valid_dataloader=valid_loader,
                                                valid=True,
                                                model=model,
                                                epochs=50,
                                                optimizer=optim.Adam,
                                                criterion=nn.CrossEntropyLoss,
                                                scheduler=optim.lr_scheduler.StepLR,
                                                gamma=0.8,
                                                step_size=4,
                                                lr=1e-03,
                                                coef=1.0,
                                                device=device)

## **(8) 학습 결과 시각화**

In [ ]:
# Accuracy 시각화

plt.figure(figsize=(12, 6))
plt.plot(train_loss)
plt.plot(valid_loss)
plt.title('Loss History')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.grid(True)

plt.tight_layout()
plt.show()